In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [2]:
#save model
save_file = "model.ckpt"

weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([4]))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print(f"weights: {sess.run(weights)}")
    print(f"bias: {sess.run(bias)}")
    
    saver.save(sess, save_file)
    



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
weights: [[ 0.6772471  -0.12965752 -1.4594178 ]
 [ 0.9900947   0.07470361  1.0910541 ]]
bias: [ 0.17668974  0.05048426  1.921876   -1.3534063 ]


In [4]:
#load model
tf.reset_default_graph()

weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([4]))

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    print(f"weights: {sess.run(weights)}")
    print(f"bias: {sess.run(bias)}")


INFO:tensorflow:Restoring parameters from model.ckpt
weights: [[ 0.6772471  -0.12965752 -1.4594178 ]
 [ 0.9900947   0.07470361  1.0910541 ]]
bias: [ 0.17668974  0.05048426  1.921876   -1.3534063 ]


In [6]:
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
tf.disable_eager_execution()

learning_rate = 0.001
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

path = os.getcwd() + '\mnist.npz'
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=path)


x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train = x_train.reshape(-1, x_train.shape[0]).T
x_test = x_test.reshape(-1, x_test.shape[0]).T

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train.astype(np.float32))
y_test = to_categorical(y_test.astype(np.float32))




print(f"x_train:{x_train.shape}")
print(f"y_train:{y_train.shape}")
print(f"x_test:{x_test.shape}")
print(f"y_test:{y_test.shape}")
y_test[:3]


x_train:(60000, 784)
y_train:(60000, 10)
x_test:(10000, 784)
y_test:(10000, 10)


array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [7]:


# Remove previous Tensors and Operations
tf.reset_default_graph()




# Import MNIST data

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Training cycle
    for epoch in range(n_epochs):
        total_batch = math.ceil(x_train.shape[0] / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_start = i*batch_size
            batch_features = x_train[batch_start:batch_start + batch_size]
            batch_labels = y_train[batch_start:batch_start + batch_size]

            sess.run(
                optimizer,
                feed_dict={features: batch_features, labels: batch_labels})

        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features:x_test,
                    labels:y_test})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.094200000166893
Epoch 10  - Validation Accuracy: 0.09740000218153
Epoch 20  - Validation Accuracy: 0.09880000352859497
Epoch 30  - Validation Accuracy: 0.09910000115633011
Epoch 40  - Validation Accuracy: 0.09920000284910202
Epoch 50  - Validation Accuracy: 0.09799999743700027
Epoch 60  - Validation Accuracy: 0.09799999743700027
Epoch 70  - Validation Accuracy: 0.09799999743700027
Epoch 80  - Validation Accuracy: 0.0982000008225441
Epoch 90  - Validation Accuracy: 0.0982000008225441
Trained Model Saved.


In [11]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: x_test, labels: y_test})

print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.0982000008225441
